# Import required packages

In [ ]:
#!pip install llama-cpp-python

In [1]:
#load packages
import pandas as pd
from functools import partial
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import json
import tiktoken
import nltk
import chromadb
from sentence_transformers import SentenceTransformer
from transformers import AutoProcessor, AutoTokenizer, AutoModelForImageTextToText, AutoModelForCausalLM, AutoModel 
import torch
from huggingface_hub import login
from dotenv import load_dotenv
import torchvision
from llama_cpp import Llama
import requests
import numpy as np
from numpy.linalg import norm

# Load in Documents

In this section we will load in the scraped documents and clean them.

In [2]:
#read in csv for majors
majors = pd.read_csv("Scraped_data/uva_majors.csv")

#remove non major data
majors = majors.iloc[8:-1]


In [3]:
# remove the extra text from the text column
test = 'Info For Students Alumni Military Affiliated Students Faculty & Staff Search Search Submit Search Close search Info For Students Alumni Military Affiliated Students Faculty & Staff Calendars Academic Calendars Exam Schedules Student Records Diplomas Transcripts FERPA Programs Calendar Schools University Registrar Carruthers Hall, 1001 N. Emmet St. P.O. Box 400203 Charlottesville, VA 22904-4203 Staff Directory Contact Info Phone: (434) 924-4122 Fax: (434) 924-4156 Email: [email protected] Hours Of Operation M-F: 10am - noon and 1pm - 4pm Â© 2024 By the Rector and Visitors of the University of Virginia Legal Privacy Report a Barrier Share Your Feedback University of Virginia Mar 31, 2025 Undergraduate Record 2024-2025 Select a Catalog Undergraduate Record 2024-2025 Graduate Record 2024-2025 Global Search Catalog Search Choose Search Location Select an option Courses Programs Schools/Colleges & Departments Policies and Other Non-Academic Content Entire Catalog Search Keyword Field Whole Word/Phrase Advanced Search Catalog Navigation Catalog Home Academic Calendar Admission Schools Programs, Degrees & Course Info Student Resources ROTC University Regulations About UVA Archived Records HELP Undergraduate Record 2024-2025'
text_to_remove_head = 'Info For Students Alumni Military Affiliated Students Faculty & Staff Search Search Submit Search Close search Info For Students Alumni Military Affiliated Students Faculty & Staff Calendars Academic Calendars Exam Schedules Student Records Diplomas Transcripts FERPA Programs Calendar Schools University Registrar Carruthers Hall, 1001 N. Emmet St. P.O. Box 400203 Charlottesville, VA 22904-4203 Staff Directory Contact Info Phone: (434) 924-4122 Fax: (434) 924-4156 Email: [email protected] Hours Of Operation M-F: 10am - noon and 1pm - 4pm Â© 2024 By the Rector and Visitors of the University of Virginia Legal Privacy Report a Barrier Share Your Feedback University of Virginia Mar 12, 2025 Undergraduate Record 2024-2025 Select a Catalog Undergraduate Record 2024-2025 Graduate Record 2024-2025 Global Search Catalog Search Choose Search Location Select an option Courses Programs Schools/Colleges & Departments Policies and Other Non-Academic Content Entire Catalog Search Keyword Field Whole Word/Phrase Advanced Search Catalog Navigation Catalog Home Academic Calendar Admission Schools Programs, Degrees & Course Info Student Resources ROTC University Regulations About UVA Archived Records HELP Undergraduate Record 2024-2025' 
text_to_remove_tail = 'Back to Top | Print-Friendly Page (opens a new window) All catalogs © 2025 University of Virginia. Powered by Modern Campus Catalog™ . .'
#function to remove the text
def remove_text(text):
    if text.startswith(text_to_remove_head):
        text = text[len(text_to_remove_head):]
    if text.startswith(test):
        text = text[len(test):]
    if text.endswith(text_to_remove_tail):
        text = text[:-len(text_to_remove_tail)]
    return text.strip()

#apply the function to the text column
majors['Text'] = majors['Text'].apply(remove_text)

#since some of the majors have extra text that is a bit different, we need to remove it
majors[majors['Major'] == 'Psychology, B.A.']['Text']

#majors[50:90]

64    Psychology, B.A. Print-Friendly Page (opens a ...
Name: Text, dtype: object

In [4]:
# remove extra text from the majors column where majors includes "interdisciplinary"
text_to_remove = 'Interdisciplinary - '
text_to_remove2 = 'Interdisciplinary Major - '
def remove_text_majors(text):
    if text.startswith(text_to_remove):
        text = text[len(text_to_remove):]
    if text.startswith(text_to_remove2):
        text = text[len(text_to_remove2):]
    return text.strip()

#apply the function to the majors column
majors['Major'] = majors['Major'].apply(remove_text_majors)

In [5]:
majors[20:40]

Major  \
28                     English, B.A.   
29      Environmental Sciences, B.A.   
30      Environmental Sciences, B.S.   
31             Foreign Affairs, B.A.   
32                      French, B.A.   
33                      German, B.A.   
34                  Government, B.A.   
35                     History, B.A.   
36                  American Studies   
37                       Archaeology   
38     Chinese Language & Literature   
39                 Cognitive Science   
40                East Asian Studies   
41  Environmental Thought & Practice   
42                    German Studies   
43                    Global Studies   
44                     Human Biology   
45    Japanese Language & Literature   
46                    Jewish Studies   
47                       Linguistics   

                                                 Text  
28  English, B.A. Print-Friendly Page (opens a new...  
29  Environmental Sciences, B.A. Print-Friendly Pa...  
30  Environmental Sciences, B.S. Print-Friendly Pa...  
31  Foreign Affairs, B.A. Print-Friendly Page (ope...  
32  French, B.A. Print-Friendly Page (opens a new ...  
33  German, B.A. Print-Friendly Page (opens a new ...  
34  Government, B.A. Print-Friendly Page (opens a ...  
35  History, B.A. Print-Friendly Page (opens a new...  
36  Interdisciplinary Major - American Studies Pri...  
37  Interdisciplinary Major - Archaeology Print-Fr...  
38  Interdisciplinary Major - Chinese Language & L...  
39  Interdisciplinary Major - Cognitive Science Pr...  
40  Interdisciplinary Major - East Asian Studies P...  
41  Interdisciplinary Major - Environmental Though...  
42  Interdisciplinary Major - German Studies Print...  
43  Interdisciplinary Major - Global Studies Print...  
44  Interdisciplinary Major - Human Biology Print-...  
45  Interdisciplinary Major - Japanese Language & ...  
46  Interdisciplinary Major - Jewish Studies Print...  
47  Interdisciplinary Major - Linguistics Print-Fr...

In [6]:
#read in schools data
schools = pd.read_csv("Scraped_data/uva_schools_academic_rules.csv")
schools

School  \
0                             School of Architecture   
1                         College of Arts & Sciences   
2                        McIntire School of Commerce   
3      School of Continuing and Professional Studies   
4                             School of Data Science   
5          School of Education and Human Development   
6          School of Engineering and Applied Science   
7  Frank Batten School of Leadership and Public P...   
8                                  School of Nursing   

                                                Text  
0  Info For Students Alumni Military Affiliated S...  
1  Info For Students Alumni Military Affiliated S...  
2  Info For Students Alumni Military Affiliated S...  
3  Info For Students Alumni Military Affiliated S...  
4  Info For Students Alumni Military Affiliated S...  
5  Info For Students Alumni Military Affiliated S...  
6  Info For Students Alumni Military Affiliated S...  
7  Info For Students Alumni Military Affiliated S...  
8  Info For Students Alumni Military Affiliated S...

In [7]:
# remove the extra text from the text column
extra_begining_text = 'Info For Students Alumni Military Affiliated Students Faculty & Staff Search Search Submit Search Close search Info For Students Alumni Military Affiliated Students Faculty & Staff Calendars Academic Calendars Exam Schedules Student Records Diplomas Transcripts FERPA Programs Calendar Schools University Registrar Carruthers Hall, 1001 N. Emmet St. P.O. Box 400203 Charlottesville, VA 22904-4203 Staff Directory Contact Info Phone: (434) 924-4122 Fax: (434) 924-4156 Email: [email protected] Hours Of Operation M-F: 10am - noon and 1pm - 4pm Â© 2024 By the Rector and Visitors of the University of Virginia Legal Privacy Report a Barrier Share Your Feedback University of Virginia Mar 31, 2025 Undergraduate Record 2024-2025 Select a Catalog Undergraduate Record 2024-2025 Graduate Record 2024-2025 Global Search Catalog Search Choose Search Location Select an option Courses Programs Schools/Colleges & Departments Policies and Other Non-Academic Content Entire Catalog Search Keyword Field Whole Word/Phrase Advanced Search Catalog Navigation Catalog Home Academic Calendar Admission Schools Programs, Degrees & Course Info Student Resources ROTC University Regulations About UVA Archived Records HELP Undergraduate Record 2024-2025'

def remove_text_requirements(text):
    if text.startswith(extra_begining_text):
        text = text[len(extra_begining_text):]
    return text.strip()

#run the function on the text column
schools['Text'] = schools['Text'].apply(remove_text_requirements)

In [8]:
schools

School  \
0                             School of Architecture   
1                         College of Arts & Sciences   
2                        McIntire School of Commerce   
3      School of Continuing and Professional Studies   
4                             School of Data Science   
5          School of Education and Human Development   
6          School of Engineering and Applied Science   
7  Frank Batten School of Leadership and Public P...   
8                                  School of Nursing   

                                                Text  
0  School of Architecture: Academic Rules, Regula...  
1  College of Arts & Sciences: Academic Rules, Re...  
2  McIntire School of Commerce: Academic Rules, R...  
3  School of Continuing and Professional Studies:...  
4  School of Data Science: Academic Rules Print-F...  
5  School of Education and Human Development: Aca...  
6  School of Engineering and Applied Science: Aca...  
7  Frank Batten School of Leadership and Public P...  
8  School of Nursing: Academic Rules, Regulations...

In [9]:
# read in class data
classes = pd.read_csv("Lous_List_Database/Fall_25_UVA_w_descr.csv")
classes.head()

ClassNumber Mnemonic Number Section        Type Units  \
0        10003      AAS   1010     100     Lecture     4   
1        12774      AAS   1010     101  Discussion     0   
2        10006      AAS   1010     102  Discussion     0   
3        10004      AAS   1010     103  Discussion     0   
4        10005      AAS   1010     104  Discussion     0   

                        Instructor                   Days  \
0  Robert Vinson, Naseemah Mohamed  TuTh 12:30pm - 1:45pm   
1                  To Be Announced     We 6:00pm - 6:50pm   
2                  To Be Announced     We 5:00pm - 5:50pm   
3                  To Be Announced     Tu 7:00pm - 7:50pm   
4                  To Be Announced     Tu 8:00pm - 8:50pm   

                  Room                                              Title  \
0       Minor Hall 125  Introduction to African-American and African S...   
1  New Cabell Hall 283  Introduction to African-American and African S...   
2  New Cabell Hall 287  Introduction to African-American and African S...   
3  New Cabell Hall 411  Introduction to African-American and African S...   
4  New Cabell Hall 411  Introduction to African-American and African S...   

  Topic Status  Enrollment  EnrollmentLimit  Waitlist  \
0   NaN   Open           0              180         0   
1   NaN   Open           0               20         0   
2   NaN   Open           0               20         0   
3   NaN   Open           0               20         0   
4   NaN   Open           0               20         0   

                                         Description  
0  This introductory course surveys the histories...  
1  This introductory course surveys the histories...  
2  This introductory course surveys the histories...  
3  This introductory course surveys the histories...  
4  This introductory course surveys the histories...

# Add school metadata to majors and convert to dictionary

In [10]:
# copy of the majors data frame
majors_meta = majors
majors_meta['School'] = None
majors_meta = majors_meta.reset_index(drop=True)

#add school to each majors in major data frame
majors_meta['School'][0:3] = 'School of Architecture'
majors_meta['School'][3:62] = 'College of Arts and Sciences'
majors_meta['School'][62:63] = 'McIntire School of Commerce'
majors_meta['School'][63:65] = 'School of Continuing and Professional Studies'
majors_meta['School'][65:66] = 'School of Data Science'
majors_meta['School'][66:72] = 'School of Education and Human Development'
majors_meta['School'][72:83] = 'School of Engineering and Applied Science'
majors_meta['School'][83:84] = 'Frank Batten School of Leadership and Public Policy'
majors_meta['School'][84:85] = 'School of Nursing'

/var/folders/42/069x05gj0h785d4_lrjy8mhw0000gn/T/ipykernel_25744/684139997.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  majors_meta['School'][0:3] = 'School of Architecture'
/var/folders/42/069x05gj0h785d4_lrjy8mhw0000gn/T/ipykernel_2

In [11]:
majors_meta.to_csv('Scraped_data/majors_meta.csv')

In [12]:
majors_meta.head(10)

Major  \
0              Architectural History, B.Ar.H.   
1                          Architecture, B.S.   
2  Urban and Environmental Planning, B.U.E.P.   
3  African-American and African Studies, B.A.   
4                          Anthropology, B.A.   
5                    Applied Statistics, B.A.   
6                          Area Studies, B.A.   
7                                 Art History   
8                                  Studio Art   
9                             Astronomy, B.A.   

                                                Text  \
0  Architectural History, B.Ar.H. Print-Friendly ...   
1  Architecture, B.S. Print-Friendly Page (opens ...   
2  Urban and Environmental Planning, B.U.E.P. Pri...   
3  African American and African Studies, B.A. Pri...   
4  Anthropology, B.A. Print-Friendly Page (opens ...   
5  Applied Statistics, B.A. Print-Friendly Page (...   
6  Area Studies, B.A. Print-Friendly Page (opens ...   
7  History of Art Print-Friendly Page (opens a ne...   
8  Studio Art Print-Friendly Page (opens a new wi...   
9  Astronomy, B.A. Print-Friendly Page (opens a n...   

                         School  
0        School of Architecture  
1        School of Architecture  
2        School of Architecture  
3  College of Arts and Sciences  
4  College of Arts and Sciences  
5  College of Arts and Sciences  
6  College of Arts and Sciences  
7  College of Arts and Sciences  
8  College of Arts and Sciences  
9  College of Arts and Sciences

In [13]:
#convert majors to dictionary
majors_dict = majors_meta.to_dict(orient='records')
majors_dict[0:2]

[{'Major': 'Architectural History, B.Ar.H.',
  'Text': 'Architectural History, B.Ar.H. Print-Friendly Page (opens a new window) Return to: School of Architecture: Degree Programs Universal Curriculum Requirements To be awarded a degree from the School of Architecture, students are required to complete universal curriculum requirements in addition to the program requirements provided below. The school universal curriculum requirements can be found on the school Degree Programs page . Program Requirements Undergraduate students entering the School of Architecture share a Common First Year in the School of Architecture . Students take core courses in Architectural History, Architecture, and Urban & Environmental Planning to provide a framework for the study of\xa0the built environment\xa0through observation, analysis, and design. Students must pass each core course with a grade of C- or higher. During the spring semester of the first year, students choose an intended major: Bachelor of Ar

In [14]:
#convert schools to dictionary
schools_dict = schools.to_dict(orient='records')
schools_dict[0:2]

[{'School': 'School of Architecture',
  'Text': 'School of Architecture: Academic Rules, Regulations, and Information Print-Friendly Page (opens a new window) About Academic Rules Degree Programs Minor Programs Courses Click on a link to be taken to the entry below. Admission Requirements Academic Standing Appeals from Students Awards for Academic Excellence Curricular Requirements Course Rules Degree Information Direct Admissions Policy Grades Intra-University Transfers Leaving and Returning to the University Majors and Minors Public Professional Licensure Disclosure Student Intellectual Property Transfer Credit Admission Requirements All prospective undergraduate students apply through the University Office of Undergraduate Admissions; http://www.admission.virginia.edu/ For general information about the School of Architecture and undergraduate admissions: https://www.arch.virginia.edu/apply/undergraduate-admissions Academic Standing Academic Good Standing Students, in the School of A

In [15]:
# dictionary for school to classes matching with Mnemonic
#query sis api
api_url = 'https://sisuva.admin.virginia.edu/psc/ihprd/UVSS/SA/s/WEBLIB_HCX_CM.H_CLASS_SEARCH.FieldFormula.IScript_ClassSearchOptions?institution=UVA01&term=1248'

r = requests.get(api_url)

sis = json.loads(r.text)
sis_df = pd.DataFrame(sis['subjects'])


In [16]:
# remove subject letters from descr column
sis_df['descr'] = sis_df.apply(lambda row: row['descr'].replace(row['subject'], ''), axis=1)
# remove extra spaces from descr column
sis_df['descr'] = sis_df['descr'].str.replace(r'- ', '', regex=True).str.replace(r'\s+', ' ', regex=True).str.strip()
sis_df.head()

subject                                    descr acad_groups acad_orgs  \
0     AAS     African-American and African Studies      [CGAS]     [AAS]   
1    ACCT                               Accounting      [SCPS]   [SCPSD]   
2    AIRS                              Air Science      [PROV]    [AIRS]   
3    ALAR  Architecture and Landscape Architecture      [ARCH]    [ALAR]   
4    AMST                         American Studies      [CGAS]    [AMST]   

        careers campuses  
0  [GRAD, UGRD]   [MAIN]  
1  [GRAD, UGRD]   [SCPS]  
2        [UGRD]   [MAIN]  
3        [GRAD]   [MAIN]  
4  [GRAD, UGRD]   [MAIN]

In [17]:
# Perform a left merge
merged_classes = classes.merge(sis_df[['subject', 'descr']], how='left', left_on='Mnemonic', right_on='subject')

# Drop the redundant 'subject' column from the merged DataFrame
merged_classes = merged_classes.drop(columns=['subject'])

# add column for semester = Fall and column for year = spring
merged_classes['semester'] = 'Fall'
merged_classes['year'] = '2025'

#replace nan description with 'none'
merged_classes['Description'] = merged_classes['Description'].fillna('none')


# Display the first few rows of the merged DataFrame
merged_classes.head()

ClassNumber Mnemonic Number Section        Type Units  \
0        10003      AAS   1010     100     Lecture     4   
1        12774      AAS   1010     101  Discussion     0   
2        10006      AAS   1010     102  Discussion     0   
3        10004      AAS   1010     103  Discussion     0   
4        10005      AAS   1010     104  Discussion     0   

                        Instructor                   Days  \
0  Robert Vinson, Naseemah Mohamed  TuTh 12:30pm - 1:45pm   
1                  To Be Announced     We 6:00pm - 6:50pm   
2                  To Be Announced     We 5:00pm - 5:50pm   
3                  To Be Announced     Tu 7:00pm - 7:50pm   
4                  To Be Announced     Tu 8:00pm - 8:50pm   

                  Room                                              Title  \
0       Minor Hall 125  Introduction to African-American and African S...   
1  New Cabell Hall 283  Introduction to African-American and African S...   
2  New Cabell Hall 287  Introduction to African-American and African S...   
3  New Cabell Hall 411  Introduction to African-American and African S...   
4  New Cabell Hall 411  Introduction to African-American and African S...   

  Topic Status  Enrollment  EnrollmentLimit  Waitlist  \
0   NaN   Open           0              180         0   
1   NaN   Open           0               20         0   
2   NaN   Open           0               20         0   
3   NaN   Open           0               20         0   
4   NaN   Open           0               20         0   

                                         Description  \
0  This introductory course surveys the histories...   
1  This introductory course surveys the histories...   
2  This introductory course surveys the histories...   
3  This introductory course surveys the histories...   
4  This introductory course surveys the histories...   

                                  descr semester  year  
0  African-American and African Studies     Fall  2025  
1  African-American and African Studies     Fall  2025  
2  African-American and African Studies     Fall  2025  
3  African-American and African Studies     Fall  2025  
4  African-American and African Studies     Fall  2025

In [18]:
#convdert classes to dictionary
classes_dict = merged_classes.to_dict(orient='records')
classes_dict[0:2]

[{'ClassNumber': 10003,
  'Mnemonic': 'AAS',
  'Number': '1010',
  'Section': '100',
  'Type': 'Lecture',
  'Units': '4',
  'Instructor': 'Robert Vinson, Naseemah Mohamed',
  'Days': 'TuTh 12:30pm - 1:45pm',
  'Room': 'Minor Hall 125',
  'Title': 'Introduction to African-American and African Studies I',
  'Topic': nan,
  'Status': 'Open',
  'Enrollment': 0,
  'EnrollmentLimit': 180,
  'Waitlist': 0,
  'Description': 'This introductory course surveys the histories of people of African descent in Africa, the Americas, and the Caribbean from approximately the Middle Ages to the 1880s. Emphases include the Atlantic slave trade and its complex relationship to Africa; the economic systems, cultures, and communities of Africans and African-Americans in the New World, in slavery and in freedom; the rise of anti-slavery movements; and the socio-economic systems that replaced slavery in the late 19th century.',
  'descr': 'African-American and African Studies',
  'semester': 'Fall',
  'year': '2

# Chunking

In [19]:
# disable tokenizer parallelism so we stop getting problems
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [20]:
nltk.download("punkt")

# Load tokenizer (for OpenAI models)
tokenizer = tiktoken.get_encoding("cl100k_base")

# Function to chunk text while maintaining sentence structure
def chunk_text(text, max_tokens=500):
    sentences = nltk.tokenize.sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        sentence_tokens = len(tokenizer.encode(sentence))
        if current_length + sentence_tokens > max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_length = 0
        current_chunk.append(sentence)
        current_length += sentence_tokens

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks



[nltk_data] Downloading package punkt to /Users/rfell/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Store data in ChromaDB vectorized database

In [ ]:
#!rm -rf ./chroma_db

In [22]:
# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db") 

In [23]:
 #create majors collection
collection = chroma_client.get_or_create_collection(name="Majors_Collection")
#create separate collection for classes
collection_classes = chroma_client.get_or_create_collection(name="Classes_Collection")

In [24]:
# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

## Don't rerun adding info to database

In [25]:
# Store chunked majors data
for course in majors_dict:
    chunks = chunk_text(course["Text"])
    for i, chunk in enumerate(chunks):
        embedding = model.encode(chunk).tolist()  # Convert to list for ChromaDB
        collection.add(
            ids=[f"{course['Major']}_chunk{i}"],  
            embeddings=[embedding],  
            metadatas=[{
                "Major": course["Major"],
                "chunk_index": i,
                "Text": chunk,
                "School": course['School']
            }]
        )

print("Chunked course descriptions stored in ChromaDB!")

Chunked course descriptions stored in ChromaDB!


In [26]:
# Store chunked schools data
for course in schools_dict:
    chunks = chunk_text(course["Text"])
    for i, chunk in enumerate(chunks):
        embedding = model.encode(chunk).tolist()  # Convert to list for ChromaDB
        collection.add(
            ids=[f"{course['School']}_chunk{i}"],  
            embeddings=[embedding],  
            metadatas=[{
                "School": course["School"],
                "chunk_index": i,
                "Text": chunk
            }]
        )

print("Chunked schools stored in ChromaDB!")

Chunked schools stored in ChromaDB!


add course number, section, semester, year, etc
this might improve


In [27]:
#function to add all the courses data together
def create_combined_text(course):
    fields_to_combine = [
        course.get("Title", ""), 
        course.get('Code', ""),
        course.get('Number', ""),
        course.get('Instructor', ""),
        course.get('Type', ""),
        course.get('Days', ""),
        course.get('semester', ""),
        course.get('year', ""),
        course.get('department', ""),
        course.get("Description", ""), 
        course.get("Instructor", ""), 
        course.get("Days", ""), 
        course.get("Room", "")
    ]
    # Join the fields with a separator (e.g., newline or space)
    combined_text = " | ".join([field for field in fields_to_combine if field])  # Skip empty fields
    return combined_text

In [28]:

#store classes in collection
for course in classes_dict:
    combined_text = create_combined_text(course)
    embedding = model.encode(combined_text).tolist()  # Convert to list for ChromaDB
    collection_classes.add(
        ids=[f"{course['ClassNumber']}_{course['Mnemonic']}_{course['Number']}_{course['Section']}"],  
        embeddings=[embedding],  
        metadatas=[{
            "Code": course["Mnemonic"],
            "Number": course["Number"],
            "Section": course["Section"],
            "Type": course["Type"],
            "Units": course["Units"],
            "Instructor": course["Instructor"],
            "Days": course["Days"],
            "Room": course["Room"],
            "Title": course["Title"],
            'Semester': course['semester'],
            'Year': course['year'], 
            "Type": 'Class',
            "Description": course["Description"],
            "Department": course["descr"]
        }]
    )

print("Classes data stored in ChromaDB!")


Classes data stored in ChromaDB!


# Retrieval

In [29]:
# function for document retrieval
#user can input the number of major documents and number of school documents they want to retrieve
def retrieve_major_and_school_info(query, k_m=10, k_s = 2):
    # Step 1: Retrieve major-related chunks
    query_embedding = model.encode(query).tolist()

    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=k_m
    )

    # Step 2: Extract major-specific results and school information
    major_info = []
    for i in range(len(results["ids"][0])):
        metadata = results["metadatas"][0][i]
        # If the metadata has 'major_name', this is a major document
        if metadata.get("Major"):
            major_info.append(metadata)

    # Step 3: Extract the school name from the major info
    schools_to_query = set([info["School"] for info in major_info])

    # Step 4: Retrieve relevant school descriptions based on the inferred school name
    school_info = []
    for school_name in schools_to_query:
        school_results = collection.query(
            query_embeddings=[query_embedding],
            where={"School": school_name},
            n_results=k_s
        )
        school_info.extend(school_results["metadatas"][0])

    return major_info, school_info

In [30]:
# Example search for "computer science"
query = "course schedule for computer science major"
major_info, school_info = retrieve_major_and_school_info(query, 10, 5)

In [31]:
major_info[0:2]

[{'Major': 'Computer Science, B.S.',
  'School': 'School of Engineering and Applied Science',
  'Text': 'The 2000 level courses should be taken before the 3000 level courses and note that there are other prerequisites that govern the order that these courses should be taken. Example schedules can be found on the Computer Science Department webpage . CS 2100\xa0-\xa0Data Structures and Algorithms 1 Credits: 4 CS 2120\xa0-\xa0Discrete Mathematics and Theory 1 Credits: 3 CS 2130\xa0-\xa0Computer Systems and Organization 1 Credits: 4 CS 3100\xa0-\xa0Data Structures and Algorithms 2 Credits:          3 CS 3120\xa0-\xa0Discrete Mathematics and Theory 2 Credits: 3 CS 3130\xa0-\xa0Computer Systems and Organization 2 Credits: 4 CS 3140\xa0-\xa0Software Development Essentials Credits: 3 Upper-Level Required Courses BSCS majors must take one upper-level course in software engineering and one course to complete the SEAS senior thesis. CS 3240\xa0-\xa0Software Engineering Credits:          3 The se

In [32]:
school_info[0:2]

[{'Major': 'Bachelor of Interdisciplinary Studies, B.I.S.',
  'School': 'School of Continuing and Professional Studies',
  'Text': 'The required Information Technology courses are: IT 3220 - Strategic Business Value of Information Technology IT 3230 - Basics of Web Design IT 3240 - Enterprise Systems Architecture & Design IT 3400 - Database Management & Business Intelligence/Analytics IT 4400 - Web Application Development - Building Dynamic Websites Elective IT Courses (3 credits) IT 3210 - Programming with Java IT 3350 - Agile Project Management IT 3600 - Data Analytics & Decision-Making SEC 3010 - Securing the Cyber World Degree Elective Courses (24-48 credits) Students in the IT concentration are also required to take twenty-four to forty-eight\xa0approved credit hours of degree electives outside of IT courses. These vary by semester and may include courses in disciplines such as anthropology, art, art history, architectural history, classics, drama, economics, history, literature, 

# Functions to transform output of retrieval to long string

In [33]:
#function to reformat major info into a long string
def format_major_info_string(major_info):
    formatted_texts = []
    for doc in major_info:
        school = doc.get('School', 'Unknown School') #get school or unknown school
        major = doc.get("Major", "Unknown Major")  # Get Major, or "Unknown Major" if missing
        text = doc.get("Text", "No Text Available")  # Get Text, or "No Text Available" if missing

        # Clean up newline characters and non-breaking spaces
        cleaned_text = text.replace("\n", " ").replace("\xa0", " ")

        formatted_texts.append(f"{school}: {major}: {cleaned_text}")

    context = "\n\n".join(formatted_texts)  # Join with double newlines for separation
    return context

In [34]:
#function to reformat school info into a long string
def format_school_info_string(school_info):
    formatted_texts = []
    for doc in school_info:
        school = doc.get('School', 'Unknown School') #get school or unknown school
        text = doc.get("Text", "No Text Available")  # Get Text, or "No Text Available" if missing

        # Clean up newline characters and non-breaking spaces
        cleaned_text = text.replace("\n", " ").replace("\xa0", " ")
        #cleaned_text = " ".join(text.split()) 

        formatted_texts.append(f"{school} \n {cleaned_text}")

    context = "\n\n".join(formatted_texts)  # Join with double newlines for separation
    return context

In [32]:
def format_class_info_string(class_info):
    formatted_texts = []
    for doc in class_info:
        code = doc.get("Code", "Unknown Code")  # Get Code, or "Unknown Code" if missing
        number = doc.get("Number", "Unknown Number")  # Get Number, or "Unknown Number" if missing
        section = doc.get("Section", "Unknown Section")  # Get Section, or "Unknown Section" if missing
        type = doc.get("Type", "Unknown Type")  # Get Type, or "Unknown Type" if missing
        units = doc.get("Units", "Unknown Units")  # Get Units, or "Unknown Units" if missing
        instructor = doc.get("Instructor", "Unknown Instructor")  # Get Instructor, or "Unknown Instructor" if missing
        days = doc.get("Days", "Unknown Days")  # Get Days, or "Unknown Days" if missing
        room = doc.get("Room", "Unknown Room")  # Get Room, or "Unknown Room" if missing
        title = doc.get("Title", "No Title Available")  # Get Title, or "No Title Available" if missing
        description = doc.get("Description", "No Description Available")  # Get Description, or "No Description Available" if missing
        semester = doc.get("Semester", "Unknown Semester")
        year = doc.get("Year", "Unknown Year")
        department = doc.get("Department", "Unknown Department")

        # Clean up newline characters and non-breaking spaces
        cleaned_description = description.replace("\n", " ").replace("\xa0", " ")

        formatted_texts.append(f"code:{code}, number: {number}, semester: {semester}, year:{year},section: {section}, units: {units}, type: {type} instructor: {instructor}, days: {days}, room: {room},department {department},  title: {title} course desrcription: {cleaned_description}")

    context = "\n\n".join(formatted_texts)  # Join with double newlines for separation
    return context

# Generation

In [33]:
# Load environment variables from .env file
load_dotenv()

# Retrieve the token
token = os.getenv("TOKEN")


In [34]:
# log into huggingface
login(token = token)

In [35]:
#set system prompt
SYSTEM_PROMPT_1 = """You are a University of Virginia undergraduate course scheduler. 
The output of this prompt will be used to search for classes, by looking for embedding similarities, in a ChromaDB vector database so make sure the output is clear, concise and under 500 tokens.
The classes database includes information on course times, descriptions, professors, and other relevant information.
As background information: You use retrieved documents to create a schedule for one semester. A full time student should have between 12-15 credits per semester. 
Do not make up any courses, only use information from the context.
Do not include any non relevant information like courses you aren't taking, or any other information that is not relevant to the course schedule.
The user will ask you to create a schedule for a specific major or school and year. Do not give a third or fourth year student all introductory classes and do not give a first year student all high level classes. The user may also provide possible additional information such as professors, days of the week, or specific courses.
"""

## Using Llamma cpp package

In [100]:
#load xxs 2-bit quantized mistral model
llm = Llama.from_pretrained(
	repo_id="bartowski/mistralai_Mistral-Small-3.1-24B-Instruct-2503-GGUF",
	filename="mistralai_Mistral-Small-3.1-24B-Instruct-2503-IQ3_XXS.gguf",
    n_ctx = 20000,
    n_gpu_layers=-1,  # Use Metal GPU acceleration
    n_threads=8,
    verbose=True 
)

llama_model_load_from_file_impl: using device Metal (Apple M3 Max) - 14370 MiB free
llama_model_loader: loaded meta data with 45 key-value pairs and 363 tensors from /Users/rfell/.cache/huggingface/hub/models--bartowski--mistralai_Mistral-Small-3.1-24B-Instruct-2503-GGUF/snapshots/6f1e0225e5ab39e4904d2ff2cc4e7805b416eff3/./mistralai_Mistral-Small-3.1-24B-Instruct-2503-IQ3_XXS.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Mistral Small 3.1 24B Instruct 2503
llama_model_loader: - kv   3:                            general.version str              = 2503
llama_model_loader: - kv   4:                           general.finetu

Other models Tried:  

original model used:  
6-bit model  
repo_id="bartowski/mistralai_Mistral-Small-3.1-24B-Instruct-2503-GGUF",  
filename="mistralai_Mistral-Small-3.1-24B-Instruct-2503-IQ2_M.gguf",  

Tried different levels of quantization: ex: IQ3_XXS instead of IQ2_M 

6-bit model: Mistral 7B (fewer parameters, returns nonesense, or just a list of 20+ classes)  
repo_id="MaziyarPanahi/Mistral-7B-Instruct-v0.3-GGUF",  
filename="Mistral-7B-Instruct-v0.3.IQ1_M.gguf  


In [37]:
#function to generate response from LLM using llama-cpp-python library
def generate_response(major_info, school_info, system_promot, query, k_m, k_s):
    '''retrieve relevant documents from majors and schools and generate response to user query using mistral 3 model'''

    #get documents
    major_info, school_info = retrieve_major_and_school_info(query, k_m, k_s)
    major = format_major_info_string(major_info)
    school = format_school_info_string(school_info)
    context = major + school

    #construct prompt
    prompt = f"{system_promot}\n\nContext:\n{context}\n\nQuestion: {query}"

    #generate response using llama-cpp-python
    response = llm(
        prompt=prompt,
        max_tokens=500,  # Adjust max tokens as needed
        temperature=0.3,  # Adjust temperature as needed
        top_p=0.5       # Adjust top_p as needed
        #stop=["\n"]      # Define stop tokens if necessary
    )

    return response["choices"][0]["text"].strip()

## Using transformers library

# 2nd Step of RAG

here I will call previous functions that retrieve major and school info, generate a response and use that tho retrieve the classes and then generate a response again.

In [50]:
SYSTEM_PROMPT_2 = """
You are a University of Virginia undergraduate course scheduler.
Use the given context to create a course schedule based on the user query.
The schedule should include corse names, course numbers, required prerequisites, course times, instructors, room, units, and any other relevant information.
Make sure  the output is clear, concise and easy to read in a table format and make sure none of the times slots overlap.
If a class has more than one time, choose one that does not overlap with other classes.
A full time student has to have between 12-15 credits per semester unless otherwise specified.
No classes should be repeated on one schedule. When it comes to courses like Special Topics, Independent Study, or Research, please highlight that this is not a standard course.
A third or fourth year student should not be given all introductory classes and a first year student should not have all high level classes.
The user may also ask for specific professors, days of the week, or specific courses.
Please use a professional tone and avoid any unnecessary information.
"""

Previously in system_promt2: The schedule should include optional courses, online/hybrid options, or specific time preferences.

In [98]:
def two_step_retrieve_and_generate_response(system_prompt1, system_prompt2, query, k_m=10, k_s=5, k_c=20):
    # Step 1: Retrieve major and school related documents
    major_info, school_info = retrieve_major_and_school_info(query, k_m, k_s)
    
    #Step 2: generate response
    response = generate_response(major_info, school_info, system_prompt1, query, k_m, k_s)


    # Step 3: Retrieve class-related documents using the initial response
    response_embedding = model.encode(response).tolist()
    class_info = []
    class_results = collection_classes.query(
        query_embeddings=[response_embedding],
        where={"Type": "Class"},  # Filter for class-related documents
        n_results=k_c
    )
    class_info = class_results["metadatas"][0]


    # Step 4: Feed class documents to the LLM along with previous
    class_context = format_class_info_string(class_info)
    final_prompt = f"{system_prompt2}\n\nMajor Context:\n{major_info}\n\nSchool Context :\n{school_info}\n\nClass Context:\n{class_context}\n\nUser Query: {query}\n\n. Generate a response."
    final_response = llm(
        prompt=final_prompt,
        max_tokens=2000,  # Adjust max tokens as needed
        temperature=0.1,  # Adjust temperature as needed
        top_p=0.5       # Adjust top_p as needed
        #stop=["\n"]      # Define stop tokens if necessary
    )

    return final_response #major_info, school_info, class_context

In [61]:
final_response, major_info, school_info, class_context = two_step_retrieve_and_generate_response(SYSTEM_PROMPT_1, SYSTEM_PROMPT_2, "Can you make a semeter schedule, for a second year biology major for fall 2025.", 10, 5, 20)

Llama.generate: 1 prefix-match hit, remaining 6303 prompt tokens to eval
llama_perf_context_print:        load time =   36611.19 ms
llama_perf_context_print: prompt eval time =   37772.47 ms /  6303 tokens (    5.99 ms per token,   166.87 tokens per second)
llama_perf_context_print:        eval time =   35365.01 ms /   499 runs   (   70.87 ms per token,    14.11 tokens per second)
llama_perf_context_print:       total time =   73317.25 ms /  6802 tokens
Llama.generate: 1 prefix-match hit, remaining 11555 prompt tokens to eval
llama_perf_context_print:        load time =   36611.19 ms
llama_perf_context_print: prompt eval time =   77998.06 ms / 11555 tokens (    6.75 ms per token,   148.14 tokens per second)
llama_perf_context_print:        eval time =   25096.81 ms /   283 runs   (   88.68 ms per token,    11.28 tokens per second)
llama_perf_context_print:       total time =  103167.44 ms / 11838 tokens


In [62]:
print(final_response["choices"][0]["text"].strip())

Here is a course schedule for a second-year Biology major for the Fall 2025 semester:

| Course Name                          | Course Number | Units | Time                | Instructor       | Room          |
|--------------------------------------|---------------|-------|--------------------|------------------|---------------|
| Introduction to Biology with Lab     | BIOL 2100     | 4     | Mo 2:00pm - 4:30pm | David Kittlesen  | Gilmer 351    |
| Introductory College Chemistry I    | CHEM 1410     | 3     | We 7:00pm - 8:30pm  | Lisa Morkowchuk  | Contact Dept  |
| Statistics for Biologists             | STAT 2020     | 4     | Fr 3:30pm - 5:00pm  | Kevin Welch      | Contact Dept  |
| Calculus I                           | MATH 1310     | 4     | Mo 9:30am - 10:45am| Kevin Welch      | Wilson 325    |

This schedule ensures that the student meets the required courses for a second-year Biology major while maintaining a manageable credit load of 15 units.


# Saved example outputs

### second year biology major fall 2025



Here is a suggested schedule for a second-year Biology major for the Fall 2025 semester:

| Course Name | Course Number | Units | Prerequisites | Time                | Instructor          | Room         |
|--------------------------------------------------|---------------|-------|----------------|---------------------|--------------------|--------------|
| Introduction to Biology w/Laboratory: Organismal & Evolutionary Biology | BIOL 2200     | 4     | None            | MoWeFr 10:00am - 10:50am | Jessamyn Manson | Chemistry Bldg 217 |
| Cell Biology | BIOL 3000     | 3     | BIOL 2100 or 2200 | TueThu 11:00am - 12:15pm | Edward Dennis      | Gilmer Hall 201 |
| Genetics and Molecular Biology                   | BIOL 3010     | 3     | BIOL 2100 or 2200 | MonWed 1:00pm - 2:15pm | Deborah McKee     | Gilmer Hall 203 |
| Evolution and Ecology                             | BIOL 3020     | 3     | BIOL 2100 or 2200 | MonWed 10:00am - 11:15am | Heather Hines      | Gilmer Hall 202 |

This schedule ensures that you complete the core requirements for the Biology major while maintaining a balanced course load. If you have any specific preferences or constraints, please let me know, and I can adjust the schedule accordingly.

Here is a suggested schedule for a second-year Biology major for the Fall 2025 semester:

| Course Name                          | Course Number | Units | Time                  | Instructor          | Room          |
|--------------------------------------|---------------|-------|-----------------------|--------------------|---------------|
| Introduction to Biology w/Lab: Org & Evol | BIOL 2200     | 4     | MoWeFr 10:00am - 10:50am | Jessamyn Manson | Chem Bldg 217 |
| |               |       | Fr 1:00pm - 3:30pm    |                    | Gilmer 353    |
| Cell Biology                          | BIOL 3000     | 3     | TuTh 10:00am - 11:15am | Michael Morris     | Gilmer 257    |
| Genetics and Molecular Biology       | BIOL 3010     | 3     | TuTh 1:30pm - 2:45pm  | Edward Griffin      | Gilmer 259    |
| Calculus I                            | MATH 1310     | 4     | MoTuWeTh 12:00pm - 12:50pm | Robert Bell | Gilmer 130    |
| Introductory College Chemistry I     | CHEM 1410     | 3     | MoWeFr 11:00am - 11:50am | John Smith         | Chemistry Bldg 203 |  |

This schedule ensures that the student meets the prerequisites for upper-level biology courses and maintains a balanced course load of 17 credits. The courses selected align with the requirements for the Biology major and provide a solid foundation for future semesters.

In [101]:
sched = two_step_retrieve_and_generate_response(SYSTEM_PROMPT_1, SYSTEM_PROMPT_2, "Can you make a semeter schedule, for a second year chemistry major for fall 2025.", 10, 5, 20)

llama_perf_context_print:        load time =   66459.03 ms
llama_perf_context_print: prompt eval time =   66457.78 ms / 10182 tokens (    6.53 ms per token,   153.21 tokens per second)
llama_perf_context_print:        eval time =    9962.40 ms /   119 runs   (   83.72 ms per token,    11.94 tokens per second)
llama_perf_context_print:       total time =   76455.29 ms / 10301 tokens
Llama.generate: 1 prefix-match hit, remaining 16408 prompt tokens to eval
llama_perf_context_print:        load time =   66459.03 ms
llama_perf_context_print: prompt eval time =  116519.13 ms / 16408 tokens (    7.10 ms per token,   140.82 tokens per second)
llama_perf_context_print:        eval time =   46900.83 ms /   439 runs   (  106.84 ms per token,     9.36 tokens per second)
llama_perf_context_print:       total time =  163578.23 ms / 16847 tokens


In [102]:
print(sched["choices"][0]["text"].strip())

Here is a schedule for a second-year Chemistry major for the Fall 2025 semester:

| Course Name                | Course Number | Units | Time                | Instructor          | Room          |
|---------------------------|---------------|-------|--------------------|--------------------|----------------|
| Introductory College Chemistry II | CHEM 1420    | 3     | Th 2:00pm - 3:15pm | Michelle Personick | Physics Bldg 242 |
| Introductory College Chemistry II Laboratory | CHEM 1421    | 1     | (To be arranged with CHEM 1420) | Michelle Personick | Chemistry Bldg 217 |
| Organic Chemistry I       | CHEM 2410    | 3     | Tu 3:30pm - 4:45pm | Kevin Welch        | Chemistry Bldg 217 |
| Organic Chemistry I Laboratory | CHEM 2411    | 3     | (To be arranged with CHEM 2410) | Kevin Welch        | Chemistry Bldg 217 |
| Principles of Physics 1 for Pre-Health Students | PHYS 2010   | 3     | Tu 12:30pm - 1:45pm | Kevin Welch        | Wilson Hall 325 |
| Principles of Physics 1 Workshop 

In [103]:
prompt = '''
You are a University of Virginia undergraduate course scheduler.
Use the given context to create a course schedule based on the user query.
The schedule should include corse names, course numbers, required prerequisites, course times, instructors, room, units, and any other relevant information.
Make sure  the output is clear, concise and easy to read in a table format and make sure none of the times slots overlap.
If a class has more than one time, choose one that does not overlap with other classes.
A full time student has to have between 12-15 credits per semester unless otherwise specified.
No classes should be repeated on one schedule. When it comes to courses like Special Topics, Independent Study, or Research, please highlight that this is not a standard course.
A third or fourth year student should not be given all introductory classes and a first year student should not have all high level classes.
The user may also ask for specific professors, days of the week, or specific courses.
Please use a professional tone and avoid any unnecessary information.
User Query: Can you make a semeter schedule, for a second year chemistry major for fall 2025.
'''
sched_no_rag = llm(
        prompt=prompt,
        max_tokens=2000,  # Adjust max tokens as needed
        temperature=0.1,  # Adjust temperature as needed
        top_p=0.5       # Adjust top_p as needed
    )

Llama.generate: 213 prefix-match hit, remaining 26 prompt tokens to eval
llama_perf_context_print:        load time =   66459.03 ms
llama_perf_context_print: prompt eval time =     246.27 ms /    26 tokens (    9.47 ms per token,   105.58 tokens per second)
llama_perf_context_print:        eval time =   69806.13 ms /  1363 runs   (   51.22 ms per token,    19.53 tokens per second)
llama_perf_context_print:       total time =   71077.91 ms /  1389 tokens


In [104]:
print(sched_no_rag["choices"][0]["text"].strip())

I would prefer to not take classes on Friday.

Here is the information for the relevant courses:

CHEM 1420: General Chemistry II
- Prerequisites: CHEM 1410
- Units: 3
- Instructors: T. B. K. Dalby
- Times: MWF 10:00AM-10:50AM, TR 8:00AM-9:20AM
- Location: Chemistry Building 235

CHEM 3310: Organic Chemistry I
- Prerequisites: CHEM 1420
- Units: 3
- Instructors: J. L. Hollis
- Times: MWF 1:00PM-1:50PM, TR 11:00AM-12:20PM
- Location: Chemistry Building 235

CHEM 3320: Organic Chemistry II
- Prerequisites: CHEM 3310
- Units: 3
- Instructors: J. L. Hollis
- Times: MWF 2:00PM-2:50PM, TR 1:00PM-2:20PM
- Location: Chemistry Building 235

CHEM 3420: Physical Chemistry I
- Prerequisites: CHEM 1420, PHYS 2010, MATH 2110
- Units: 3
- Instructors: E. M. Armentrout
- Times: MWF 11:00AM-11:50AM, TR 9:30AM-10:50AM
- Location: Chemistry Building 235

CHEM 3430: Physical Chemistry II
- Prerequisites: CHEM 3420, MATH 2210
- Units: 3
- Instructors: E. M. Armentrout
- Times: MWF 12:00PM-12:50PM, TR 11:00

I would prefer to not take classes on Friday.

Here is the information for the relevant courses:

CHEM 1420: General Chemistry II
- Prerequisites: CHEM 1410
- Units: 3
- Instructors: T. B. K. Dalby
- Times: MWF 10:00AM-10:50AM, TR 8:00AM-9:20AM
- Location: Chemistry Building 235

CHEM 3310: Organic Chemistry I
- Prerequisites: CHEM 1420
- Units: 3
- Instructors: J. L. Hollis
- Times: MWF 1:00PM-1:50PM, TR 11:00AM-12:20PM
- Location: Chemistry Building 235

CHEM 3320: Organic Chemistry II
- Prerequisites: CHEM 3310
- Units: 3
- Instructors: J. L. Hollis
- Times: MWF 2:00PM-2:50PM, TR 1:00PM-2:20PM
- Location: Chemistry Building 235

CHEM 3420: Physical Chemistry I
- Prerequisites: CHEM 1420, PHYS 2010, MATH 2110
- Units: 3
- Instructors: E. M. Armentrout
- Times: MWF 11:00AM-11:50AM, TR 9:30AM-10:50AM
- Location: Chemistry Building 235

CHEM 3430: Physical Chemistry II
- Prerequisites: CHEM 3420, MATH 2210
- Units: 3
- Instructors: E. M. Armentrout
- Times: MWF 12:00PM-12:50PM, TR 11:00AM-12:20PM
- Location: Chemistry Building 235

CHEM 3950: Special Topics in Chemistry
- Units: 3
- Instructors: Varies
- Times: MWF 3:00PM-3:50PM, TR 2:30PM-3:50PM
- Location: Chemistry Building 235

MATH 2110: Calculus II
- Prerequisites: MATH 1210
- Units: 4
- Instructors: A. B. C. Dull
- Times: MWF 9:00AM-9:50AM, TR 8:00AM-9:20AM
- Location: Monroe Hall 125

MATH 2210: Calculus III
- Prerequisites: MATH 2110
- Units: 4
- Instructors: A. B. C. Dull
- Times: MWF 10:00AM-10:50AM, TR 9:30AM-10:50AM
- Location: Monroe Hall 125

PHYS 2010: Physics II
- Prerequisites: PHYS 2010
- Units: 4
- Instructors: E. V. Dewey
- Times: MWF 11:00AM-11:50AM, TR 12:00PM-1:20PM
- Location: Small Hall 310

PHYS 2020: Physics II
- Prerequisites: PHYS 2010
- Units: 4
- Instructors: E. V. Dewey
- Times: MWF 12:00PM-12:50PM, TR 1:00PM-2:20PM
- Location: Small Hall 310

Here is the information for the relevant courses that are not part of the chemistry major but are required for the major:

MATH 1210: Calculus I
- Units: 4
- Instructors: A. B. C. Dull
- Times: MWF 8:00AM-8:50AM, TR 8:00AM-9:20AM
- Location: Monroe Hall 125

PHYS 2010: Physics I
- Units: 4
- Instructors: E. V. Dewey
- Times: MWF 9:00AM-9:50AM, TR 10:00AM-11:20AM
- Location: Small Hall 310

Based on this information, here is a suggested schedule for a second year chemistry major for fall 2025, avoiding Friday classes:

| Course       | Course Number | Units | Instructor   | Time               | Location     |
|--------------|---------------|-------|--------------|--------------------|--------------|
| General Chemistry II | CHEM 1420  | 3     | T. B. K. Dalby | MWF 10:00AM-10:50AM | Chem 235    |
| Organic Chemistry I | CHEM 3310  | 3     | J. L. Hollis  | TR 11:00AM-12:20PM | Chem 235    |
| Calculus II  | MATH 2110    | 4     | A. B. C. Dull | TR 8:00AM-9:20AM    | Monroe 125  |
| Physics I    | PHYS 2010    | 4     | E. V. Dewey   | MWF 9:00AM-9:50AM   | Small 310   |

This schedule totals 14 units, which is within the recommended range for a full-time student.

# prompt options

Can you make a semester schedule for a second year biomedical engineering student for fall 2025?

Can you make a semester schedule or a fourth year biomedical engineering student for fall 2025?

Can you make a semester schedule for a first year economics student for fall 2025?

Can you make a semester schedule for a third year economics student for fall 2025?

Can you make a semester schedule for a third year architecture student for fall 2025?




# Tests

### Can you make a semester schedule for a second year biomedical engineering student for fall 2025?

In [73]:
bio2, major_bio2, school_bio2, class_bio2 = two_step_retrieve_and_generate_response(SYSTEM_PROMPT_1, SYSTEM_PROMPT_2, "Can you make a semester schedule for a second year biomedical engineering student for fall 2025?.", 10, 5, 20)

Llama.generate: 222 prefix-match hit, remaining 8487 prompt tokens to eval
llama_perf_context_print:        load time =   36611.19 ms
llama_perf_context_print: prompt eval time =   78603.37 ms /  8488 tokens (    9.26 ms per token,   107.99 tokens per second)
llama_perf_context_print:        eval time =   39894.00 ms /   499 runs   (   79.95 ms per token,    12.51 tokens per second)
llama_perf_context_print:       total time =   94034.83 ms /  8987 tokens
Llama.generate: 1 prefix-match hit, remaining 12781 prompt tokens to eval
llama_perf_context_print:        load time =   36611.19 ms
llama_perf_context_print: prompt eval time =   86061.36 ms / 12781 tokens (    6.73 ms per token,   148.51 tokens per second)
llama_perf_context_print:        eval time =   40255.12 ms /   429 runs   (   93.83 ms per token,    10.66 tokens per second)
llama_perf_context_print:       total time =  126462.63 ms / 13210 tokens


In [74]:
print(bio2["choices"][0]["text"].strip())

Here is a schedule for a second-year Biomedical Engineering student for the Fall 2025 semester:

| Course Name                          | Course Number | Units | Time                | Instructor            | Room              | Prerequisites |
|--------------------------------------|---------------|-------|---------------------|-----------------------|--------------------|--|
| Multivariable Calculus                | APMA 2120    | 4     | TuTh 11:00am - 12:15pm | Anne Fernando        | Chemical Eng Bldg 005 | None       |
| Introduction to Programming          | CS 1110       | 3     | MoWeFr 2:00pm - 2:50pm | Gary Koenig           | Mechanical Eng Bldg 339 | None       |
| Engineering Foundations 2            | ENGR 1020     | 3     | To be announced      | To be announced      | To be announced    | ENGR 1010 |
| Introductory Physics 1 for Engineers  | PHYS 1425     | 3     | MoWeFr 10:00am - 10:50am | Jency Sundararajan    | Physics Bldg 238    | None       |
| Introductory Physics

### Can you make a semester schedule or a fourth year biomedical engineering student for fall 2025?

In [79]:
final_bio4, major_bio4, school_bio4, class_bio4 = two_step_retrieve_and_generate_response(SYSTEM_PROMPT_1, SYSTEM_PROMPT_2, "Can you make a semester schedule or a fourth year biomedical engineering student for fall 2025?", 10, 5, 20)

Llama.generate: 1 prefix-match hit, remaining 8350 prompt tokens to eval
llama_perf_context_print:        load time =   36611.19 ms
llama_perf_context_print: prompt eval time =   53075.64 ms /  8350 tokens (    6.36 ms per token,   157.32 tokens per second)
llama_perf_context_print:        eval time =   38686.61 ms /   499 runs   (   77.53 ms per token,    12.90 tokens per second)
llama_perf_context_print:       total time =   91933.42 ms /  8849 tokens
Llama.generate: 1 prefix-match hit, remaining 12359 prompt tokens to eval
llama_perf_context_print:        load time =   36611.19 ms
llama_perf_context_print: prompt eval time =   82691.54 ms / 12359 tokens (    6.69 ms per token,   149.46 tokens per second)
llama_perf_context_print:        eval time =   29438.62 ms /   316 runs   (   93.16 ms per token,    10.73 tokens per second)
llama_perf_context_print:       total time =  112221.86 ms / 12675 tokens


In [80]:
print(final_bio4["choices"][0]["text"].strip())

Here is a schedule for a fourth-year Biomedical Engineering student for Fall 2025:

| Course Name                          | Course Number | Units | Time                | Instructor            | Room               |
|--------------------------------------|---------------|-------|---------------------|-----------------------|--------------------|
| Biomedical Engineering Capstone Design I | BME-4063     | 3     | TuTh 9:30am - 10:45am | Timothy Allen        | PINN Hall 1005     |
| Biomedical Engineering Advanced Projects | BME-4995     | 1-3   | TBA                 | (Choose one instructor)| TBA                |
| Special Topics in Biomedical Engineering | BME-4550     | 3     | MoWe 2:00pm - 3:15pm | Brian Helmke         | Biomed Engr & Med Sci 1041 |
| STS and Engineering Practice          | STS-4500     | 3     | (Time TBA)          | (Instructor TBA)      | (Room TBA)         |

**Notes:**
- **Biomedical Engineering Advanced Projects**: Choose one instructor from the available opti

### Can you make a semester schedule for a first year economics student for fall 2025?


In [82]:
final_eco1, major_eco1, school_eco1, class_eco1 = two_step_retrieve_and_generate_response(SYSTEM_PROMPT_1, SYSTEM_PROMPT_2, "Can you make a semester schedule for a first year economics student for fall 2025?", 10, 5, 20)

Llama.generate: 1 prefix-match hit, remaining 9741 prompt tokens to eval
llama_perf_context_print:        load time =   36611.19 ms
llama_perf_context_print: prompt eval time =   62325.23 ms /  9741 tokens (    6.40 ms per token,   156.29 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   62326.37 ms /  9742 tokens
Llama.generate: 1 prefix-match hit, remaining 13868 prompt tokens to eval
llama_perf_context_print:        load time =   36611.19 ms
llama_perf_context_print: prompt eval time =   95171.89 ms / 13868 tokens (    6.86 ms per token,   145.72 tokens per second)
llama_perf_context_print:        eval time =   31744.07 ms /   328 runs   (   96.78 ms per token,    10.33 tokens per second)
llama_perf_context_print:       total time =  127012.25 ms / 14196 tokens


In [83]:
print(final_eco1["choices"][0]["text"].strip())

Here is a suggested schedule:

| Course Name | Course Number | Units | Time | Instructor | Room |
|---|---|---|---|---|---|
| Principles of Economics: Microeconomics | ECON 2010 | 3 | M W 10:00 AM - 11:15 AM | To Be Announced | Monroe Hall 101 |
| Principles of Economics: Macroeconomics | ECON 2020 | 3 | T Th 11:00 AM - 12:15 PM | To Be Announced | Monroe Hall 102 |
| Introduction to Statistical Analysis | STAT 2120 | 4 | T Th 1:00 PM - 2:15 PM | To Be Announced | Monroe Hall 103 |
| Engaging Aesthetics | EGMT 1510 | 2 | MoWe 3:30 PM - 4:45 PM | To Be Announced | Dell 2 103 |
| Artistic, Interpretive, & Philosophical Inquiry | AIPI 1000 | 3 | TuTh 2:30 PM - 3:45 PM | To Be Announced | Monroe Hall 104 |

This schedule ensures that the student meets the core requirements for the Economics major while also fulfilling general education requirements. The courses are designed to provide a balanced load of 14 credits, which is within the recommended range for a first-year student.


### Can you make a semester schedule for a third year economics student for fall 2025?

In [84]:
final_eco3, major_eco3, school_eco3, class_eco3 = two_step_retrieve_and_generate_response(SYSTEM_PROMPT_1, SYSTEM_PROMPT_2, "Can you make a semester schedule for a third year economics student for fall 2025?", 10, 5, 20)

Llama.generate: 1 prefix-match hit, remaining 9797 prompt tokens to eval
llama_perf_context_print:        load time =   36611.19 ms
llama_perf_context_print: prompt eval time =   63534.81 ms /  9797 tokens (    6.49 ms per token,   154.20 tokens per second)
llama_perf_context_print:        eval time =   40300.21 ms /   486 runs   (   82.92 ms per token,    12.06 tokens per second)
llama_perf_context_print:       total time =  104001.72 ms / 10283 tokens
Llama.generate: 1 prefix-match hit, remaining 14593 prompt tokens to eval
llama_perf_context_print:        load time =   36611.19 ms
llama_perf_context_print: prompt eval time =  102743.14 ms / 14593 tokens (    7.04 ms per token,   142.03 tokens per second)
llama_perf_context_print:        eval time =   34908.30 ms /   332 runs   (  105.15 ms per token,     9.51 tokens per second)
llama_perf_context_print:       total time =  137753.22 ms / 14925 tokens


In [85]:
print(final_eco3["choices"][0]["text"].strip())

Here is a schedule for a third-year Economics student for Fall 2025:

| Course Name                          | Course Number | Units | Time                | Instructor          | Room          |
|---------------------------------------|---------------|-------|---------------------|--------------------|---------------|
| Principles of Economics: Microeconomics | ECON 2010     | 3     | Tu 8:00am - 8:50am  | To Be Announced    | Wilson 214    |
| Principles of Economics: Macroeconomics | ECON 2020     | 3     | Th 7:00pm - 7:50pm  | To Be Announced    | Chemistry 306 |
| Intermediate Microeconomics           | ECON 3010     | 4     | Mo 2:00pm - 3:15pm | To Be Announced    | Monroe 101    |
| Introduction to Econometrics           | ECON 3720     | 4     | Tu 1:00pm - 2:15pm | To Be Announced    | Monroe 102    |
| Intermediate Macroeconomics          | ECON 3020     | 3     | Fr 1:00pm - 1:50pm | To Be Announced    | Monroe 103    |

This schedule ensures that the student meets the core

### Can you make a semester schedule for a third year architecture student for fall 2025?

In [86]:
final_ar, major_ar, school_ar, class_ar = two_step_retrieve_and_generate_response(SYSTEM_PROMPT_1, SYSTEM_PROMPT_2, "Can you make a semester schedule for a third year architecture student for fall 2025?", 10, 5, 20)

Llama.generate: 1 prefix-match hit, remaining 7147 prompt tokens to eval
llama_perf_context_print:        load time =   36611.19 ms
llama_perf_context_print: prompt eval time =   43715.22 ms /  7147 tokens (    6.12 ms per token,   163.49 tokens per second)
llama_perf_context_print:        eval time =   21097.80 ms /   289 runs   (   73.00 ms per token,    13.70 tokens per second)
llama_perf_context_print:       total time =   64893.99 ms /  7436 tokens
Llama.generate: 1 prefix-match hit, remaining 10596 prompt tokens to eval
llama_perf_context_print:        load time =   36611.19 ms
llama_perf_context_print: prompt eval time =   69201.04 ms / 10596 tokens (    6.53 ms per token,   153.12 tokens per second)
llama_perf_context_print:        eval time =   21085.96 ms /   248 runs   (   85.02 ms per token,    11.76 tokens per second)
llama_perf_context_print:       total time =   90353.75 ms / 10844 tokens


In [87]:
print(final_ar["choices"][0]["text"].strip())

Here is a schedule for a third-year Architecture student for the Fall 2025 semester:

| Course Name                  | Course Number | Units | Time                | Instructor          | Room            |
|------------------------------|---------------|-------|---------------------|--------------------|------------------|
| Foundation Studio I          | ARCH 6010    | 6     | Wed 1:00pm - 1:50pm | To Be Announced    | Campbell Hall 401 |
| History of Architecture I    | ARH 1010     | 3     | Mon 10:00am - 10:50am, Wed 10:00am - 10:50am | Lisa Reilly | Campbell Hall 153 |
| Special Topics in Architecture| ARCH 5500    | 3     | Wed 9:00am - 11:30am | JT Bachman         | Campbell Hall 305 |

This schedule ensures that the student meets the required credits for a full-time student (12-15 units) and avoids any time conflicts.


# Test calculations

## Answer Relevence
Give outputs to ChatGPT with the following prompt:  


Generate a question for the given answer.  
answer:   

In [97]:
model = SentenceTransformer("all-MiniLM-L6-v2")

#get embeddings
def get_embedding(text):
    embedding = model.encode(text)

    return embedding

#calculate cosine_similarity
def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))


In [94]:
def get_sim_score(user_query, generated_question):
    emb_q = get_embedding(user_query)
    emb_qi = get_embedding(generated_question)

    sim_score = cosine_similarity(emb_q, emb_qi)

    return print(f"User Query: {user_query}\n Generated Question: {generated_question}\nSimilarity score: {sim_score:.4f}\n -------------")

In [95]:
#querys to test
querys = ['Can you make a semester schedule for a second year biomedical engineering student for fall 2025?',
          'Can you make a semester schedule or a fourth year biomedical engineering student for fall 2025?',
          'Can you make a semester schedule for a first year economics student for fall 2025?',
          'Can you make a semester schedule for a third year economics student for fall 2025?',
          'Can you make a semester schedule for a third year architecture student for fall 2025?']

#generated responses generated with output and ask to "generate a question for the given answer"
gen_responses= ['What would a sample Fall 2025 semester schedule look like for a second-year Biomedical Engineering student, including courses, times, instructors, and prerequisites?',
                'What would be an appropriate course schedule for a fourth-year Biomedical Engineering student in Fall 2025?',
                'What is a recommended course schedule for a first-year student pursuing an Economics major while meeting general education requirements?',
                'What is a recommended Fall 2025 course schedule for a third-year Economics major that fulfills core requirements and maintains a balanced credit load?',
                'What is a suitable Fall 2025 course schedule for a third-year Architecture student that meets full-time credit requirements and avoids scheduling conflicts?'
]



In [96]:
for i,query in enumerate(querys):
    get_sim_score(query, gen_responses[i])

User Query: Can you make a semester schedule for a second year biomedical engineering student for fall 2025?
 Generated Question: What would a sample Fall 2025 semester schedule look like for a second-year Biomedical Engineering student, including courses, times, instructors, and prerequisites?
Similarity score: 0.8313
 -------------
User Query: Can you make a semester schedule or a fourth year biomedical engineering student for fall 2025?
 Generated Question: What would be an appropriate course schedule for a fourth-year Biomedical Engineering student in Fall 2025?
Similarity score: 0.8420
 -------------
User Query: Can you make a semester schedule for a first year economics student for fall 2025?
 Generated Question: What is a recommended course schedule for a first-year student pursuing an Economics major while meeting general education requirements?
Similarity score: 0.6874
 -------------
User Query: Can you make a semester schedule for a third year economics student for fall 2025?